In [24]:
import os
import boto3
from dotenv import load_dotenv
import io 
import pandas as pd

load_dotenv()

YC_ACCESS_KEY_ID = os.getenv("YC_ACCESS_KEY_ID")
YC_SECRET_ACCESS_KEY = os.getenv("YC_SECRET_ACCESS_KEY")
YC_ENDPOINT_URL = os.getenv("YC_ENDPOINT_URL")
YC_BUCKET_NAME = os.getenv("YC_BUCKET_NAME")

In [25]:
session = boto3.session.Session()
s3_client = session.client(
    service_name='s3',
    endpoint_url=YC_ENDPOINT_URL,
    aws_access_key_id=YC_ACCESS_KEY_ID,
    aws_secret_access_key=YC_SECRET_ACCESS_KEY
)

In [26]:
import gzip

file_names = ["item_categories.csv.gzip", "items.csv.gzip", "sample_submission.csv.gzip", "shops.csv.gzip", "test.csv.gzip", "sales_train.csv.gzip"]
data_location = "compressed_data/"

data_storage = dict()
for file_name in file_names:
    response = s3_client.get_object(Bucket=YC_BUCKET_NAME, Key=f'{data_location}{file_name}')
    data_storage[file_name[: len(file_name) - 9]] = pd.read_csv(io.BytesIO(response['Body'].read()), compression='gzip')


train = data_storage["sales_train"]

In [27]:

sales           = data_storage['sales_train']
items           = data_storage['items']
item_categories = data_storage['item_categories']
shops           = data_storage['shops']
test            = data_storage['test']

In [28]:
sales['date'] = pd.to_datetime(sales['date'], format = '%d.%m.%Y')
# exclude shops not in test
sales = sales[sales['shop_id'].isin(test['shop_id'].unique())]

In [29]:
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [30]:
# remove outliers 
sales = sales[ (sales['item_price'] < 100000 )]
sales = sales[ (sales['item_cnt_day'] < 1001)]
sales = sales[ (sales['item_price'] > 0 )]
sales = sales[ (sales['item_cnt_day'] > 0)]

In [23]:
'item_price' in sales

True

In [31]:
sales['revenue'] = sales['item_price']*sales['item_cnt_day']

In [32]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn import metrics
import lightgbm as lgb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [33]:
# cerate test-like train - add rows for all shops&items&periods product. For combinations not in original data fill 0
from tqdm import tqdm
def create_testlike_train(df):
    matrix = []
    min_date = df['date'].min()
    for i in tqdm(range(df['date_block_num'].min(), df['date_block_num'].max()+1)):
        shops = df[df['date_block_num'] == i]['shop_id'].unique()
        items = df[df['date_block_num'] == i]['item_id'].unique()
        month_start = min_date + pd.tseries.offsets.DateOffset(months = i)
        matrix.append( np.array( list(product([i],[month_start],shops,items))))
    df_new = pd.DataFrame(np.vstack(matrix),columns = ['date_block_num','month_start','shop_id','item_id'])
    pivot = pd.pivot_table(df, 
                            values = ['item_cnt_day','revenue'], 
                            index = ['date_block_num','shop_id','item_id'], 
                            aggfunc = 'sum').reset_index()
    pivot2 = pd.pivot_table(df[df['item_cnt_day']>0], 
                            values = ['item_cnt_day'], 
                            index = ['date_block_num','shop_id','item_id'], 
                            aggfunc = 'count').reset_index()
    pivot2.rename(columns={'item_cnt_day': 'purch_cnt_month'}, inplace=True)
    
    df_new = df_new.merge(right = pivot, how = 'left', on = ['date_block_num','shop_id','item_id'], sort = False)
    df_new = df_new.merge(right = pivot2, how = 'left', on = ['date_block_num','shop_id','item_id'], sort = False)
    
    df_new.rename(columns={'item_cnt_day': 'item_cnt_month_uncl'}, inplace=True)
    df_new['item_cnt_month_uncl'] = df_new['item_cnt_month_uncl'].fillna(0)
    df_new['item_cnt_month'] = df_new['item_cnt_month_uncl'].clip(0,20)
    df_new['revenue'] = df_new['revenue'].fillna(0)
    df_new['purch_cnt_month'] = df_new['purch_cnt_month'].fillna(0)
    df_new['ID'] = -1
    return df_new

In [34]:
%%time
df = create_testlike_train(sales)

100%|██████████| 34/34 [00:02<00:00, 11.90it/s]


CPU times: user 12.6 s, sys: 3.16 s, total: 15.7 s
Wall time: 15.7 s


In [35]:
df

,date_block_num,month_start,shop_id,item_id,item_cnt_month_uncl,revenue,purch_cnt_month,item_cnt_month,ID
0,0,2013-01-01,59,22154,1.0,999.0,1.0,1.0,-1
1,0,2013-01-01,59,2552,0.0,0.0,0.0,0.0,-1
2,0,2013-01-01,59,2554,0.0,0.0,0.0,0.0,-1
3,0,2013-01-01,59,2555,0.0,0.0,0.0,0.0,-1
4,0,2013-01-01,59,2564,0.0,0.0,0.0,0.0,-1
...,...,...,...,...,...,...,...,...,...
8593725,33,2015-10-01,21,7635,0.0,0.0,0.0,0.0,-1
8593726,33,2015-10-01,21,7638,0.0,0.0,0.0,0.0,-1
8593727,33,2015-10-01,21,7640,0.0,0.0,0.0,0.0,-1
8593728,33,2015-10-01,21,7632,0.0,0.0,0.0,0.0,-1


In [36]:
test['date_block_num'] = df['date_block_num'].max()+1
test['month_start'] = df['month_start'].max() + pd.tseries.offsets.DateOffset(months = 1)
test['item_cnt_month'] = 0
test['item_cnt_month_uncl'] = 0
test['revenue'] = 0
test['purch_cnt_month'] = 0

#test['sh_it_key'] = test['shop_id'].astype(str) + ['-']*len(test['shop_id']) + test['item_id'].astype(str)
#train_key = list(set(sales['shop_id'].astype(str) + ['-']*len(sales['shop_id']) + sales['item_id'].astype(str)))
#test['was_in_s_it_sh'] = test['sh_it_key'].apply(lambda x: 1 if x in train_key else 0)
#test.drop('sh_it_key', inplace=True, axis=1)

In [37]:
# concat train and test to a single df
df = df[['ID','date_block_num','month_start','shop_id','item_id','item_cnt_month_uncl','item_cnt_month','purch_cnt_month','revenue']]
test = test[['ID','date_block_num','month_start','shop_id','item_id','item_cnt_month_uncl','item_cnt_month','purch_cnt_month','revenue']]
df = pd.concat([df,test], ignore_index=True, join = 'inner')
#del(test)
df.shape

(8807930, 9)

In [38]:

df['ID'] = df['ID'].astype('int32')
df['date_block_num'] = df['date_block_num'].astype('int8')
df['shop_id'] = df['shop_id'].astype('int8')
df['item_id'] = df['item_id'].astype('int16')
df['item_cnt_month'] = df['item_cnt_month'].astype('float32')
df['item_cnt_month_uncl'] = df['item_cnt_month_uncl'].astype('float32')
df['revenue'] = df['revenue'].astype('float32')
df['purch_cnt_month'] = df['purch_cnt_month'].astype('float32')
#df['was_in_s_it_sh'] = df['was_in_s_it_sh'].astype('int8')

In [39]:
#add months and days in month features
df['month'] = df['month_start'].dt.month.astype('int8')
#df['year'] = df['month_start'].dt.year.astype('int16')
df.drop(['month_start'], axis = 1, inplace = True)

days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
df['days_in_m'] = (df['month']-1).map(days).astype('int8')

In [40]:
df

,ID,date_block_num,shop_id,item_id,item_cnt_month_uncl,item_cnt_month,purch_cnt_month,revenue,month,days_in_m
0,-1,0,59,22154,1.0,1.0,1.0,999.0,1,31
1,-1,0,59,2552,0.0,0.0,0.0,0.0,1,31
2,-1,0,59,2554,0.0,0.0,0.0,0.0,1,31
3,-1,0,59,2555,0.0,0.0,0.0,0.0,1,31
4,-1,0,59,2564,0.0,0.0,0.0,0.0,1,31
...,...,...,...,...,...,...,...,...,...,...
8807925,214195,34,45,18454,0.0,0.0,0.0,0.0,11,30
8807926,214196,34,45,16188,0.0,0.0,0.0,0.0,11,30
8807927,214197,34,45,15757,0.0,0.0,0.0,0.0,11,30
8807928,214198,34,45,19648,0.0,0.0,0.0,0.0,11,30


In [41]:
# add city and type features
shops['shop_city'] = shops['shop_name'].apply(lambda x: x.split()[0])
shops['shop_type'] = shops['shop_name'].apply(lambda x: x.split()[1])

In [42]:
# add item categories features
item_categories['split'] = item_categories['item_category_name'].str.split('-')
item_categories['item_category_type'] = item_categories['split'].map(lambda x: x[0].strip())
item_categories['item_category_subtype'] = item_categories['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
item_categories.drop('split', axis = 1, inplace = True)

df = df.merge(items, 
              how='left', 
              on='item_id').merge(item_categories, 
                                  how ='left', 
                                  on='item_category_id').merge(shops, how = 'left', on='shop_id')

df['item_category_id'] = df['item_category_id'].astype('int8')

In [43]:
df

,ID,date_block_num,shop_id,item_id,item_cnt_month_uncl,item_cnt_month,purch_cnt_month,revenue,month,days_in_m,item_name,item_category_id,item_category_name,item_category_type,item_category_subtype,shop_name,shop_city,shop_type
0,-1,0,59,22154,1.0,1.0,1.0,999.0,1,31,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино,Blu,"Ярославль ТЦ ""Альтаир""",Ярославль,ТЦ
1,-1,0,59,2552,0.0,0.0,0.0,0.0,1,31,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Музыка,Винил,"Ярославль ТЦ ""Альтаир""",Ярославль,ТЦ
2,-1,0,59,2554,0.0,0.0,0.0,0.0,1,31,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,Музыка,Винил,"Ярославль ТЦ ""Альтаир""",Ярославль,ТЦ
3,-1,0,59,2555,0.0,0.0,0.0,0.0,1,31,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,Музыка,CD фирменного производства,"Ярославль ТЦ ""Альтаир""",Ярославль,ТЦ
4,-1,0,59,2564,0.0,0.0,0.0,0.0,1,31,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео,Музыка,Музыкальное видео,"Ярославль ТЦ ""Альтаир""",Ярославль,ТЦ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8807925,214195,34,45,18454,0.0,0.0,0.0,0.0,11,30,СБ. Союз 55,55,Музыка - CD локального производства,Музыка,CD локального производства,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ
8807926,214196,34,45,16188,0.0,0.0,0.0,0.0,11,30,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,Подарки,Настольные игры,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ
8807927,214197,34,45,15757,0.0,0.0,0.0,0.0,11,30,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,Музыка,CD локального производства,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ
8807928,214198,34,45,19648,0.0,0.0,0.0,0.0,11,30,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,Кино,DVD,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ


In [44]:
# encode categorical features
features_to_encode=['shop_city',
                    'shop_type',
                    'item_category_type',
                    'item_category_subtype']
def encode_cat_features(df,features_to_encode):
    for feat in features_to_encode:
        df[feat+'_encoded'] = LabelEncoder().fit_transform( df[feat] )
    df.drop(features_to_encode, axis = 1, inplace = True)
    return df

df = encode_cat_features(df,features_to_encode)

df.drop(['item_category_name','shop_name','item_name'], axis = 1, inplace = True)

In [45]:
df

,ID,date_block_num,shop_id,item_id,item_cnt_month_uncl,item_cnt_month,purch_cnt_month,revenue,month,days_in_m,item_category_id,shop_city_encoded,shop_type_encoded,item_category_type_encoded,item_category_subtype_encoded
0,-1,0,59,22154,1.0,1.0,1.0,999.0,1,31,37,27,8,11,1
1,-1,0,59,2552,0.0,0.0,0.0,0.0,1,31,58,27,8,13,27
2,-1,0,59,2554,0.0,0.0,0.0,0.0,1,31,58,27,8,13,27
3,-1,0,59,2555,0.0,0.0,0.0,0.0,1,31,56,27,8,13,3
4,-1,0,59,2564,0.0,0.0,0.0,0.0,1,31,59,27,8,13,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8807925,214195,34,45,18454,0.0,0.0,0.0,0.0,11,30,55,18,8,13,2
8807926,214196,34,45,16188,0.0,0.0,0.0,0.0,11,30,64,18,8,14,42
8807927,214197,34,45,15757,0.0,0.0,0.0,0.0,11,30,55,18,8,13,2
8807928,214198,34,45,19648,0.0,0.0,0.0,0.0,11,30,40,18,8,11,4


In [20]:
# это пока скипнул

# add features for months since shop, item, item&shop first and last sale

def add_col_months_from_sh_it_last_s(df):
    vect_months_from_last_s = []
    dict_ = {}
    for ind, row in df.iterrows():
        key = str(row['shop_id']) + ' ' + str(row['item_id'])
        if key not in dict_:
            if row['item_cnt_month'] > 0:
                dict_[key] = row['date_block_num']
                vect_months_from_last_s.append(0)
            else:
                vect_months_from_last_s.append(0)
        else:
            last_b_1 = dict_[key]
            last = row['date_block_num']
            vect_months_from_last_s.append(last-last_b_1)
            dict_[key] = row['date_block_num']
    df['months_from_sh_it_last_s'] = vect_months_from_last_s
    df['months_from_sh_it_last_s'] = df['months_from_sh_it_last_s'].astype('int16')
    return df

def add_col_months_from_it_last_s(df):
    vect_months_from_last_s = []
    dict_ = {}
    dict_2 = {}
    for ind, row in df.iterrows():
        key = str(row['item_id'])
        if key not in dict_:
            if row['item_cnt_month'] > 0:
                dict_[key] = row['date_block_num']
                dict_2[key] = row['date_block_num']
                vect_months_from_last_s.append(0)
            else:
                vect_months_from_last_s.append(0)
        else:
            last_b_1 = dict_[key]
            last = row['date_block_num']
            if last > last_b_1:      
                vect_months_from_last_s.append(last-last_b_1)
                dict_2[key] = last_b_1
                dict_[key] = row['date_block_num']
            elif last == last_b_1:
                last_b_1 = dict_2[key]
                vect_months_from_last_s.append(last-last_b_1)          
    df['months_from_it_last_s'] = vect_months_from_last_s
    df['months_from_it_last_s'] = df['months_from_it_last_s'].astype('int16')
    return df

def add_col_months_since_it_first_s(df):
    pivot = pd.pivot_table(df[df['item_cnt_month']>0], values = 'date_block_num', index = 'item_id', aggfunc = 'min').reset_index()
    pivot.rename(columns={'date_block_num': 'dt_block_it_first_sale'}, inplace=True)
    df =  df.merge(right = pivot, how = 'left', on = 'item_id', sort = False) 
#    df['is_it_first_s'] = (df['date_block_num'] == df['dt_block_it_first_sale']).astype('int8')
#    df['it_had_sales_before'] = (df['date_block_num'] > df['dt_block_it_first_sale']).astype('int8').fillna(0)
#    df['months_since_it_first_s'] = (df['date_block_num'] - df['dt_block_it_first_sale']).fillna(0)
#    df['months_since_it_first_s'] = df['months_since_it_first_s'].astype('int16')
    df['dt_block_it_first_sale'] = df['dt_block_it_first_sale'].fillna(34).astype('int16')
    df['months_since_it_first_s'] = df['date_block_num'] - df['dt_block_it_first_sale']
    df.drop('dt_block_it_first_sale', inplace=True, axis=1)
    return df

def add_col_months_since_sh_it_first_sale(df):
    pivot = pd.pivot_table(df[df['item_cnt_month']>0], values = 'date_block_num', index = ['shop_id','item_id'], aggfunc = 'min').reset_index()
    pivot.rename(columns={'date_block_num': 'dt_block_sh_it_first_sale'}, inplace=True)
    df =  df.merge(right = pivot, how = 'left', on = ['shop_id','item_id'], sort = False)
#    df['is_sh_it_first_s'] = (df['date_block_num'] == df['dt_block_sh_it_first_sale']).astype('int8')
#    df['sh_it_had_sales_before'] = (df['date_block_num'] > df['dt_block_sh_it_first_sale']).astype('int8').fillna(0)
#    df['months_since_sh_it_first_s'] = (df['date_block_num'] - df['dt_block_sh_it_first_sale']).fillna(0)
#    df['months_since_sh_it_first_s'] = df['months_since_sh_it_first_s'].astype('int16')
    df['dt_block_sh_it_first_sale'] = df['dt_block_sh_it_first_sale'].fillna(34).astype('int16')
    df['months_since_sh_it_first_s'] = df['date_block_num'] - df['dt_block_sh_it_first_sale']
    df.drop('dt_block_sh_it_first_sale', inplace=True, axis=1)
    return df

def add_col_months_since_sh_first_s(df):
    pivot = pd.pivot_table(df[df['item_cnt_month']>0], values = 'date_block_num', index = 'shop_id', aggfunc = 'min').reset_index()
    pivot.rename(columns={'date_block_num': 'dt_block_sh_first_sale'}, inplace=True)
    df =  df.merge(right = pivot, how = 'left', on = 'shop_id', sort = False) 
#    df['is_sh_first_s'] = (df['date_block_num'] == df['dt_block_sh_first_sale']).astype('int8')
#    df['sh_had_sales_before'] = (df['date_block_num'] > df['dt_block_sh_first_sale']).astype('int8').fillna(0)
#    df['months_since_sh_first_s'] = (df['date_block_num'] - df['dt_block_sh_first_sale']).fillna(0)
#    df['months_since_sh_first_s'] = df['months_since_sh_first_s'].astype('int16')
    df['dt_block_sh_first_sale'] = df['dt_block_sh_first_sale'].fillna(34).astype('int16')
    df['months_since_sh_first_s'] = df['date_block_num'] - df['dt_block_sh_first_sale']
    df.drop('dt_block_sh_first_sale', inplace=True, axis=1)
    return df

In [21]:
%%time
df = add_col_months_from_sh_it_last_s(df)
df = add_col_months_from_it_last_s(df)

CPU times: user 3min 15s, sys: 713 ms, total: 3min 16s
Wall time: 3min 16s


In [22]:
%%time
df = add_col_months_since_it_first_s(df)
df = add_col_months_since_sh_it_first_sale(df)
df = add_col_months_since_sh_first_s(df)

CPU times: user 1.65 s, sys: 327 ms, total: 1.98 s
Wall time: 1.98 s


In [23]:
# это спипнул
# add mean encoded features
def add_mean_encoded_feat(df):
    pivot_it = pd.pivot_table(df, values = ['item_cnt_month',], index = ['item_id','date_block_num'], aggfunc = ['sum','count']).reset_index()
    pivot_it.columns = ['item_id','date_block_num','item_cnt_month_sum','item_cnt_month_cnt']
    pivot_it['lagged_it_mean'] = ((pivot_it.groupby(['item_id'])['item_cnt_month_sum'].cumsum() - pivot_it['item_cnt_month_sum'])/(pivot_it.groupby(['item_id'])['item_cnt_month_cnt'].cumsum() - pivot_it['item_cnt_month_cnt'])).fillna(0)
    pivot_it.drop(['item_cnt_month_sum','item_cnt_month_cnt'], axis = 1, inplace = True)
    df =  df.merge(right = pivot_it, how = 'left', on = ['item_id','date_block_num'], sort = False)
    
    pivot_sh_it = pd.pivot_table(df, values = 'item_cnt_month', index = ['shop_id','item_id','date_block_num'], aggfunc = 'sum').reset_index()
    pivot_sh_it['lagged_sh_it_mean'] = ((pivot_sh_it.groupby(['shop_id','item_id'])['item_cnt_month'].cumsum() - pivot_sh_it['item_cnt_month'])/(pivot_sh_it.groupby(['shop_id','item_id'])['item_cnt_month'].cumcount())).fillna(0)
    pivot_sh_it.drop(['item_cnt_month'], axis = 1, inplace = True)
    df =  df.merge(right = pivot_sh_it, how = 'left', on = ['shop_id','item_id','date_block_num'], sort = False)
    df['lagged_it_mean'] = df['lagged_it_mean'].astype('float32')
    df['lagged_sh_it_mean'] = df['lagged_sh_it_mean'].astype('float32')
    return df

In [24]:
%%time
df = add_mean_encoded_feat(df)

CPU times: user 5.27 s, sys: 1.1 s, total: 6.37 s
Wall time: 6.37 s


In [25]:
#add lag features
def add_lag_feat(df, col_to_agg, group_levels, n_lags, aggfunc = 'mean', clip = False):
    new_col_title_code = '_'.join([x for x in group_levels if x != 'date_block_num'])
    pivot = pd.pivot_table(df, values = col_to_agg, index = group_levels, aggfunc = aggfunc).reset_index()        
    pivot.rename(columns = {col_to_agg[0] : col_to_agg[0] + '_' + aggfunc}, inplace = True)
    idx_cols = ['date_block_num','shop_id','item_id']
    cols = list(set(idx_cols+group_levels))
    df_tech = df[cols].copy()
    df_tech = df_tech.merge(right = pivot, how = 'left', on = group_levels, sort = False)
    list_of_new_col = [] 
    for lag in n_lags:
        df_to_shift = df_tech[idx_cols+[col_to_agg[0] + '_' + aggfunc]].copy()
        df_to_shift['date_block_num'] = df_to_shift['date_block_num'] + lag
        df_to_shift.rename(columns={col_to_agg[0] + '_' + aggfunc : col_to_agg[0]+'_'+new_col_title_code + '_' + aggfunc+ '_lag_'+str(lag)}, inplace=True)
        list_of_new_col.append(col_to_agg[0]+'_'+new_col_title_code + '_' + aggfunc+ '_lag_'+str(lag))
        df= df.merge(right = df_to_shift, how = 'left', on = idx_cols, sort = False)
    for col in list_of_new_col:
        df[col] = df[col].fillna(0).astype('float32')
        if clip:
            df[col] = df[col].clip(0,20)
    return df

In [26]:
%%time
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','shop_id','item_id'], 
                  n_lags = [1,2,3], 
                  aggfunc = 'sum')

df = add_lag_feat(df, 
                  col_to_agg = ['purch_cnt_month'],
                  group_levels = ['date_block_num','shop_id','item_id'], 
                  n_lags = [1,2], 
                  aggfunc = 'sum')

CPU times: user 18.1 s, sys: 4.04 s, total: 22.2 s
Wall time: 22.2 s


In [27]:
# это тоже скип

df['it_cnt_sh_it_lag_avg'] = df[['item_cnt_month_shop_id_item_id_sum_lag_1', 
                                 'item_cnt_month_shop_id_item_id_sum_lag_2', 
                                 'item_cnt_month_shop_id_item_id_sum_lag_3']].mean(skipna=True, axis=1)

df['it_cnt_sh_it_lag_grad'] = df['item_cnt_month_shop_id_item_id_sum_lag_1']/df['item_cnt_month_shop_id_item_id_sum_lag_2']

df['it_cnt_sh_it_lag_avg'] = df['it_cnt_sh_it_lag_avg'].astype('float32')
df['it_cnt_sh_it_lag_grad'] = df['it_cnt_sh_it_lag_grad'].replace([np.inf, -np.inf], np.nan).fillna(0).astype('float32')

In [28]:
%%time
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num'], 
                  n_lags = [1])

df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','item_id'], 
                  n_lags = [1,2])
                   
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','shop_id'], 
                  n_lags = [1,2])
# df = add_lag_feat(df, 
#                   col_to_agg = ['revenue'],
#                   group_levels = ['date_block_num','shop_id'], 
#                   n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','item_category_id'], 
                  n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','shop_id','item_category_id'], 
                  n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','shop_id','item_category_type_encoded'], 
                  n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','shop_id','item_category_subtype_encoded'], 
                  n_lags = [1])
# df = add_lag_feat(df, 
#                   col_to_agg = ['item_cnt_month'],
#                   group_levels = ['date_block_num','shop_city_encoded'], 
#                   n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','shop_city_encoded','item_id'], 
                  n_lags = [1])
# df = add_lag_feat(df, 
#                   col_to_agg = ['item_cnt_month'],
#                   group_levels = ['date_block_num','shop_city_encoded','item_category_id'], 
#                   n_lags = [1])
# df = add_lag_feat(df, 
#                   col_to_agg = ['item_cnt_month'],
#                   group_levels = ['date_block_num','shop_city_encoded','item_category_type_encoded'], 
#                   n_lags = [1])
# df = add_lag_feat(df, 
#                   col_to_agg = ['item_cnt_month'],
#                   group_levels = ['date_block_num','shop_city_encoded','item_category_subtype_encoded'], 
#                   n_lags = [1])
# df = add_lag_feat(df, 
#                   col_to_agg = ['item_cnt_month'],
#                   group_levels = ['date_block_num','shop_type_encoded'], 
#                   n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','shop_type_encoded','item_id'], 
                  n_lags = [1])
# df = add_lag_feat(df, 
#                   col_to_agg = ['item_cnt_month'],
#                   group_levels = ['date_block_num','shop_type_encoded','item_category_subtype_encoded'], 
#                   n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','item_category_type_encoded'], 
                  n_lags = [1])
df = add_lag_feat(df, 
                  col_to_agg = ['item_cnt_month'],
                  group_levels = ['date_block_num','item_category_subtype_encoded'], 
                  n_lags = [1])

CPU times: user 36.8 s, sys: 9.71 s, total: 46.6 s
Wall time: 46.7 s


In [29]:
# add price lag features
def add_price_lag_feat(df):
    pivot_global = pd.pivot_table(df[(df['item_cnt_month_uncl']>0)|(df['date_block_num']==34)], 
                                  values = ['item_cnt_month_uncl','revenue'], 
                                  index = ['item_id'], 
                                  aggfunc = 'sum').reset_index()
    pivot_global['avg_price_global'] = pivot_global['revenue']/pivot_global['item_cnt_month_uncl']
    pivot_global.drop(['item_cnt_month_uncl','revenue'],axis = 1,inplace = True)
    
    pivot_avg_mnth = pd.pivot_table(df[(df['item_cnt_month_uncl']>0)|(df['date_block_num']==34)], 
                                    values = ['item_cnt_month_uncl','revenue'], 
                                    index = ['item_id','date_block_num'], 
                                    aggfunc = 'sum').reset_index()
    pivot_avg_mnth['item_id'] = pivot_avg_mnth['item_id'].astype('int16')
    pivot_avg_mnth['date_block_num'] = pivot_avg_mnth['date_block_num'].astype('int8')
    pivot_avg_mnth['avg_price_mnth'] = pivot_avg_mnth['revenue']/pivot_avg_mnth['item_cnt_month_uncl']
    pivot_avg_mnth['avg_price_mnth_lag1'] = pivot_avg_mnth.groupby(['item_id'])['avg_price_mnth'].shift(1)
    pivot_avg_mnth['avg_price_mnth_lag2'] = pivot_avg_mnth.groupby(['item_id'])['avg_price_mnth'].shift(2)
    pivot_avg_mnth.drop(['item_cnt_month_uncl','revenue','avg_price_mnth'],axis = 1,inplace = True)
    
    pivot_avg_mnth_sh = pd.pivot_table(df[(df['item_cnt_month_uncl']>0)|(df['date_block_num']==34)], 
                                       values = ['item_cnt_month_uncl','revenue'], 
                                        index = ['shop_id','item_id','date_block_num'], 
                                       aggfunc = 'sum').reset_index()
    pivot_avg_mnth_sh['shop_id'] = pivot_avg_mnth_sh['shop_id'].astype('int8')
    pivot_avg_mnth_sh['item_id'] = pivot_avg_mnth_sh['item_id'].astype('int16')
    pivot_avg_mnth_sh['date_block_num'] = pivot_avg_mnth_sh['date_block_num'].astype('int8')
    pivot_avg_mnth_sh['avg_price_mnth_sh'] = pivot_avg_mnth_sh['revenue']/pivot_avg_mnth_sh['item_cnt_month_uncl']
    pivot_avg_mnth_sh['avg_price_mnth_sh_lag1'] = pivot_avg_mnth_sh.groupby(['shop_id','item_id'])['avg_price_mnth_sh'].shift(1)
    pivot_avg_mnth_sh['avg_price_mnth_sh_lag2'] = pivot_avg_mnth_sh.groupby(['shop_id','item_id'])['avg_price_mnth_sh'].shift(2)
    pivot_avg_mnth_sh.drop(['item_cnt_month_uncl','revenue','avg_price_mnth_sh'],axis = 1,inplace = True)
    
    pivot_avg_mnth.sort_values(by = 'date_block_num', inplace = True)
    pivot_avg_mnth_sh.sort_values(by = 'date_block_num', inplace = True)
    
    df = df.merge(right = pivot_global, how = 'left', on = ['item_id'], sort = False)
    df = pd.merge_asof(left = df, 
                       right = pivot_avg_mnth, 
                       on = ['date_block_num'], 
                       by = ['item_id'], 
                       allow_exact_matches = True, 
                       direction ='backward')
    df = pd.merge_asof(left = df, 
                       right = pivot_avg_mnth_sh, 
                       on = ['date_block_num'], 
                       by = ['shop_id','item_id'], 
                       allow_exact_matches = True, 
                       direction ='backward')
    
    df['avg_price_global'] = df['avg_price_global'].replace([np.inf, -np.inf], np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_lag1'] = df['avg_price_mnth_lag1'].replace([np.inf, -np.inf], np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_lag2'] = df['avg_price_mnth_lag2'].replace([np.inf, -np.inf], np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_sh_lag1'] = df['avg_price_mnth_sh_lag1'].replace([np.inf, -np.inf], np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_sh_lag2'] = df['avg_price_mnth_sh_lag2'].replace([np.inf, -np.inf], np.nan).fillna(0).astype('float32')
    
    df['avg_price_mnth_grad'] = (df['avg_price_mnth_lag1']/df['avg_price_mnth_lag2']).replace([np.inf, -np.inf], 
                                                                                             np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_to_gl'] = (df['avg_price_mnth_lag1']/df['avg_price_global']).replace([np.inf, -np.inf], 
                                                                                             np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_sh_grad'] = (df['avg_price_mnth_sh_lag1']/df['avg_price_mnth_sh_lag2']).replace([np.inf, -np.inf], 
                                                                                             np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_sh_to_gl'] = (df['avg_price_mnth_sh_lag1']/df['avg_price_global']).replace([np.inf, -np.inf], 
                                                                                             np.nan).fillna(0).astype('float32')
#     df['avg_price_mnth_sh_to_gl_2'] = (df['avg_price_mnth_sh_lag2']/df['avg_price_global']).replace([np.inf, -np.inf], 
#                                                                                              np.nan).fillna(0).astype('float32')
    df['avg_price_mnth_sh_to_mnth'] = (df['avg_price_mnth_sh_lag1']/df['avg_price_mnth_lag1']).replace([np.inf, -np.inf], 
                                                                                             np.nan).fillna(0).astype('float32')
#     df['avg_price_mnth_sh_to_mnth_2'] = (df['avg_price_mnth_sh_lag2']/df['avg_price_mnth_lag2']).replace([np.inf, -np.inf], 
#                                                                                              np.nan).fillna(0).astype('float32')
#    df.drop(['avg_price_global','avg_price_mnth_lag2','avg_price_mnth_sh_lag2'], axis = 1, inplace = True)
    return df

In [30]:
%%time
df = add_price_lag_feat(df)

CPU times: user 3.46 s, sys: 3.28 s, total: 6.74 s
Wall time: 6.74 s


In [31]:
df = df[(df['month']!=12)&(df['month']!=1)]

In [32]:
columns_to_exclude = ['ID',
                      'item_cnt_month',
                      'item_cnt_month_uncl',
                      'revenue',
                      'purch_cnt_month',
                      'months_since_sh_it_first_s',
                      'months_since_it_first_s',
                      'months_since_sh_first_s', 
                      'avg_price_global', 
#                      'avg_price_mnth_lag1', 
#                      'avg_price_mnth_lag2',
#                      'avg_price_mnth_sh_lag1', 
#                      'avg_price_mnth_sh_lag2'
#                      'avg_price_mnth_to_gl', 
#                      'avg_price_mnth_sh_to_gl', 
#                      'months_from_sh_it_last_s',
#                      'months_from_it_last_s',
                      'lagged_sh_it_mean',
                      'lagged_it_mean',
#                      'it_had_sales_before'
#                      'sh_it_had_sales_before'
                     ]
cat_features = ['month',
#                'year',
                'shop_id',
                'shop_city_encoded',
                'shop_type_encoded',
                'item_category_id',
                'item_category_type_encoded',
                'item_category_subtype_encoded',
                'days_in_m'
               ]

In [33]:
# modeling
params = {'metric': 'rmse',
          'objective': 'mse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise' : True,
          'random_state': 10}

# Prepare training and validation datasets with categorical features
train_data = lgb.Dataset(
    df[(df['date_block_num'] >= 19) & (df['date_block_num'] < 33)].drop(columns_to_exclude, axis=1),
    label=df[(df['date_block_num'] >= 19) & (df['date_block_num'] < 33)]['item_cnt_month'],
    categorical_feature=cat_features
)

valid_data = lgb.Dataset(
    df[df['date_block_num'] == 33].drop(columns_to_exclude, axis=1),
    label=df[df['date_block_num'] == 33]['item_cnt_month'],
    categorical_feature=cat_features,
    reference=train_data
)

# Train model
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    num_boost_round=1500,
    valid_sets=[train_data, valid_data],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(100)]
)


[LightGBM] [Info] Total Bins 5968
[LightGBM] [Info] Number of data points in the train set: 2722499, number of used features: 41
[LightGBM] [Info] Start training from score 0.283914
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.92969	valid_1's rmse: 0.906929
[200]	training's rmse: 0.814937	valid_1's rmse: 0.83173
[300]	training's rmse: 0.754779	valid_1's rmse: 0.801989
[400]	training's rmse: 0.720642	valid_1's rmse: 0.791554
[500]	training's rmse: 0.698602	valid_1's rmse: 0.788439
[600]	training's rmse: 0.683074	valid_1's rmse: 0.788294
Early stopping, best iteration is:
[547]	training's rmse: 0.69101	valid_1's rmse: 0.787691
